In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna import Trial, visualization
from hepml.core import plot_regression_tree
import torch
from torch.optim import Adam, SGD
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.autograd import Variable
import torch.optim as optim
import random
import xgboost as xg
import pickle
import os
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold

In [3]:
train_df =  pd.read_csv('n_train.csv')
test_df  =  pd.read_csv('n_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
year
quarter
month
week
dayofyear
season
loc_group
climateregions__climateregion


In [4]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 260


In [5]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(seed=42)

In [7]:
cols1 = ['loc_group', 'climateregions__climateregion', 'month']
region_col = 'climateregions__climateregion'
important_col = 'contest-wind-h500-14d__wind-hgt-500'
main_cols = [target[0], important_col]

In [8]:
def create_lag_variables(df):
    for lag in tqdm([1, 7, 14]):
        for col in tqdm(main_cols):
            df[f'lag_{col}_{lag}'] = df[cols1 + [col]].groupby(cols1).shift(lag)
    return df

In [9]:
columns = main_attrs + target

In [10]:
lag_train_df = create_lag_variables(train_df[columns])

100%|████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:37<00:00, 32.59s/it]


In [12]:
lag_cols = ['lag_contest-wind-h500-14d__wind-hgt-500_1',
       'lag_contest-tmp2m-14d__tmp2m_7',
       'lag_contest-wind-h500-14d__wind-hgt-500_7',
       'lag_contest-tmp2m-14d__tmp2m_14',
       'lag_contest-wind-h500-14d__wind-hgt-500_14']

In [13]:
train_df[lag_cols] = lag_train_df[lag_cols]

In [22]:
test_df=test_df.merge(toto1,on=cols1,how="left")

In [23]:
test_df.head()

,startdate,year,quarter,month,week,dayofyear,season,day_of_year_sin,day_of_year_cos,week_sin,week_cos,month_sin,month_cos,season_sin,season_cos,quarter_sin,quarter_cos,lat,lon,loc_group,climateregions__climateregion,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,ice

In [25]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 265


In [27]:
best_cols = ['contest-wind-h500-14d__wind-hgt-500', 'contest-slp-14d__slp',
       'nmme-tmp2m-34w__ccsm3', 'elevation__elevation', 'lon',
       'contest-prwtr-eatm-14d__prwtr', 'lat', 'climateregions__climateregion',
       'contest-pres-sfc-gauss-14d__pres', 'season_sin', 'day_of_year_sin',
       'contest-precip-14d__precip',
       'contest-wind-uwnd-250-14d__wind-uwnd-250', 'nmme-prate-34w__cfsv2',
       'nmme-prate-34w__nasa', 'nmme-prate-56w__gfdlflora',
       'wind-uwnd-250-2010-7', 'contest-wind-vwnd-925-14d__wind-vwnd-925',
       'nmme-prate-34w__nmmemean', 'nmme0-prate-34w__ccsm30',
       'contest-wind-h850-14d__wind-hgt-850',
       'contest-wind-uwnd-925-14d__wind-uwnd-925', 'nmme0-prate-56w__cfsv20',
       'nmme-prate-34w__cancm3', 'contest-rhum-sig995-14d__rhum',
       'nmme-prate-34w__gfdlflorb', 'wind-hgt-850-2010-4',
       'contest-wind-vwnd-250-14d__wind-vwnd-250', 'wind-hgt-100-2010-2',
       'wind-uwnd-250-2010-18', 'wind-hgt-10-2010-5', 'wind-uwnd-250-2010-15',
       'wind-uwnd-250-2010-4', 'nmme0-prate-56w__nasa0',
       'nmme0-prate-34w__cfsv20', 'wind-vwnd-250-2010-10',
       'contest-wind-h10-14d__wind-hgt-10', 'wind-uwnd-925-2010-15',
       'wind-vwnd-250-2010-3', 'nmme-prate-34w__cancm4', 'sst-2010-4',
       'nmme0-prate-56w__ccsm30', 'wind-uwnd-250-2010-16',
       'nmme0-prate-34w__gfdl0', 'nmme0-prate-56w__cancm40', 'sst-2010-1',
       'sst-2010-3', 'wind-uwnd-250-2010-14', 'nmme0-prate-34w__gfdlflora0',
       'nmme-prate-34w__gfdl', 'wind-hgt-850-2010-9', 'wind-vwnd-250-2010-1',
       'sst-2010-5', 'cancm30', 'nmme-prate-34w__ccsm4',
       'nmme0-prate-34w__nasa0', 'wind-hgt-500-2010-9',
       'nmme0-prate-34w__cancm30', 'wind-vwnd-250-2010-13']

In [28]:
features = best_cols + lag_cols

In [29]:
train = train_df[features + target]
test = test_df[features]

In [30]:
X = train[features].values
y = train[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])   

5it [00:10,  2.16s/it]


In [31]:
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
sample_df = pd.read_csv('./sample_solution.csv')
sample_df['contest-tmp2m-14d__tmp2m'] = s
sample_df.to_csv('submission_lag_v1.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

(375734, 64) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]Warning: less than 75% gpu memory available for training. Free: 7278.5625 Total: 24268.3125


Learning rate set to 0.072699
0:	learn: 9.2547047	test: 9.2336182	best: 9.2336182 (0)	total: 23.6ms	remaining: 1m 57s
1:	learn: 8.6824653	test: 8.6617897	best: 8.6617897 (1)	total: 28ms	remaining: 1m 9s
2:	learn: 8.1545314	test: 8.1352330	best: 8.1352330 (2)	total: 32.1ms	remaining: 53.5s
3:	learn: 7.6603612	test: 7.6411100	best: 7.6411100 (3)	total: 36.1ms	remaining: 45.1s
4:	learn: 7.2008575	test: 7.1822608	best: 7.1822608 (4)	total: 40.1ms	remaining: 40.1s
5:	learn: 6.7804925	test: 6.7626584	best: 6.7626584 (5)	total: 44.5ms	remaining: 37s
6:	learn: 6.3899588	test: 6.3731962	best: 6.3731962 (6)	total: 48.6ms	remaining: 34.7s
7:	learn: 6.0274368	test: 6.0114145	best: 6.0114145 (7)	total: 56.6ms	remaining: 35.3s
8:	learn: 5.6909352	test: 5.6767718	best: 5.6767718 (8)	total: 63.4ms	remaining: 35.2s
9:	learn: 5.3796660	test: 5.3668383	best: 5.3668383 (9)	total: 67.9ms	remaining: 33.9s
10:	learn: 5.0858404	test: 5.0737974	best: 5.0737974 (10)	total: 71.8ms	remaining: 32.6s
11:	learn: 4.8

121:	learn: 1.2512823	test: 1.2595580	best: 1.2595580 (121)	total: 632ms	remaining: 25.3s
122:	learn: 1.2476281	test: 1.2558548	best: 1.2558548 (122)	total: 636ms	remaining: 25.2s
123:	learn: 1.2436372	test: 1.2519616	best: 1.2519616 (123)	total: 640ms	remaining: 25.2s
124:	learn: 1.2407475	test: 1.2489772	best: 1.2489772 (124)	total: 645ms	remaining: 25.1s
125:	learn: 1.2378332	test: 1.2461191	best: 1.2461191 (125)	total: 652ms	remaining: 25.2s
126:	learn: 1.2345775	test: 1.2429016	best: 1.2429016 (126)	total: 659ms	remaining: 25.3s
127:	learn: 1.2324849	test: 1.2408999	best: 1.2408999 (127)	total: 663ms	remaining: 25.2s
128:	learn: 1.2287126	test: 1.2370633	best: 1.2370633 (128)	total: 667ms	remaining: 25.2s
129:	learn: 1.2267397	test: 1.2349864	best: 1.2349864 (129)	total: 671ms	remaining: 25.1s
130:	learn: 1.2234486	test: 1.2317987	best: 1.2317987 (130)	total: 675ms	remaining: 25.1s
131:	learn: 1.2215057	test: 1.2298854	best: 1.2298854 (131)	total: 679ms	remaining: 25.1s
132:	learn

246:	learn: 0.9941066	test: 1.0044002	best: 1.0044002 (246)	total: 1.23s	remaining: 23.7s
247:	learn: 0.9929961	test: 1.0033035	best: 1.0033035 (247)	total: 1.25s	remaining: 23.9s
248:	learn: 0.9911826	test: 1.0014773	best: 1.0014773 (248)	total: 1.26s	remaining: 24s
249:	learn: 0.9884704	test: 0.9988429	best: 0.9988429 (249)	total: 1.26s	remaining: 24s
250:	learn: 0.9874344	test: 0.9977784	best: 0.9977784 (250)	total: 1.26s	remaining: 23.9s
251:	learn: 0.9857108	test: 0.9961351	best: 0.9961351 (251)	total: 1.27s	remaining: 23.9s
252:	learn: 0.9846727	test: 0.9951265	best: 0.9951265 (252)	total: 1.27s	remaining: 23.9s
253:	learn: 0.9834997	test: 0.9939730	best: 0.9939730 (253)	total: 1.28s	remaining: 23.9s
254:	learn: 0.9826340	test: 0.9930556	best: 0.9930556 (254)	total: 1.28s	remaining: 23.9s
255:	learn: 0.9810850	test: 0.9914039	best: 0.9914039 (255)	total: 1.29s	remaining: 23.8s
256:	learn: 0.9797646	test: 0.9901152	best: 0.9901152 (256)	total: 1.29s	remaining: 23.8s
257:	learn: 0.

...

4724:	learn: 0.3320986	test: 0.3616403	best: 0.3616403 (4724)	total: 22s	remaining: 1.28s
4725:	learn: 0.3320685	test: 0.3616203	best: 0.3616203 (4725)	total: 22s	remaining: 1.27s
4726:	learn: 0.3320454	test: 0.3615965	best: 0.3615965 (4726)	total: 22s	remaining: 1.27s
4727:	learn: 0.3320200	test: 0.3615724	best: 0.3615724 (4727)	total: 22s	remaining: 1.26s
4728:	learn: 0.3319956	test: 0.3615493	best: 0.3615493 (4728)	total: 22s	remaining: 1.26s
4729:	learn: 0.3319714	test: 0.3615305	best: 0.3615305 (4729)	total: 22s	remaining: 1.25s
4730:	learn: 0.3319187	test: 0.3614814	best: 0.3614814 (4730)	total: 22s	remaining: 1.25s
4731:	learn: 0.3318872	test: 0.3614551	best: 0.3614551 (4731)	total: 22s	remaining: 1.25s
4732:	learn: 0.3318449	test: 0.3614072	best: 0.3614072 (4732)	total: 22s	remaining: 1.24s
4733:	learn: 0.3318319	test: 0.3613984	best: 0.3613984 (4733)	total: 22s	remaining: 1.24s
4734:	learn: 0.3318046	test: 0.3613816	best: 0.3613816 (4734)	total: 22s	remaining: 1.23s
4735:	lear

4814:	learn: 0.3293038	test: 0.3591066	best: 0.3591066 (4814)	total: 22.4s	remaining: 859ms
4815:	learn: 0.3292886	test: 0.3590954	best: 0.3590954 (4815)	total: 22.4s	remaining: 854ms
4816:	learn: 0.3292648	test: 0.3590679	best: 0.3590679 (4816)	total: 22.4s	remaining: 850ms
4817:	learn: 0.3292361	test: 0.3590451	best: 0.3590451 (4817)	total: 22.4s	remaining: 845ms
4818:	learn: 0.3292032	test: 0.3590110	best: 0.3590110 (4818)	total: 22.4s	remaining: 840ms
4819:	learn: 0.3291716	test: 0.3589830	best: 0.3589830 (4819)	total: 22.4s	remaining: 836ms
4820:	learn: 0.3291406	test: 0.3589413	best: 0.3589413 (4820)	total: 22.4s	remaining: 831ms
4821:	learn: 0.3291024	test: 0.3589007	best: 0.3589007 (4821)	total: 22.4s	remaining: 827ms
4822:	learn: 0.3290839	test: 0.3588869	best: 0.3588869 (4822)	total: 22.4s	remaining: 822ms
4823:	learn: 0.3290471	test: 0.3588449	best: 0.3588449 (4823)	total: 22.4s	remaining: 817ms
4824:	learn: 0.3290201	test: 0.3588196	best: 0.3588196 (4824)	total: 22.4s	remai

4944:	learn: 0.3257038	test: 0.3557168	best: 0.3557168 (4944)	total: 23s	remaining: 255ms
4945:	learn: 0.3256807	test: 0.3557003	best: 0.3557003 (4945)	total: 23s	remaining: 251ms
4946:	learn: 0.3256595	test: 0.3556862	best: 0.3556862 (4946)	total: 23s	remaining: 246ms
4947:	learn: 0.3256376	test: 0.3556678	best: 0.3556678 (4947)	total: 23s	remaining: 241ms
4948:	learn: 0.3256164	test: 0.3556502	best: 0.3556502 (4948)	total: 23s	remaining: 237ms
4949:	learn: 0.3255995	test: 0.3556386	best: 0.3556386 (4949)	total: 23s	remaining: 232ms
4950:	learn: 0.3255805	test: 0.3556207	best: 0.3556207 (4950)	total: 23s	remaining: 227ms
4951:	learn: 0.3255439	test: 0.3555814	best: 0.3555814 (4951)	total: 23s	remaining: 223ms
4952:	learn: 0.3255153	test: 0.3555577	best: 0.3555577 (4952)	total: 23s	remaining: 218ms
4953:	learn: 0.3254563	test: 0.3555087	best: 0.3555087 (4953)	total: 23s	remaining: 214ms
4954:	learn: 0.3254202	test: 0.3554769	best: 0.3554769 (4954)	total: 23s	remaining: 209ms
4955:	lear

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.29it/s]


cos_sim with best submission: 0.997710294766568


In [33]:
sample_df = pd.read_csv('./sample_solution.csv')
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
t= s * 0.1 + y_best *0.9
sample_df['contest-tmp2m-14d__tmp2m'] = t
sample_df.to_csv('submission_lag_v1.csv', index=False)


from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, t)/(norm(y_best)*norm(t))
print("cos_sim with best submission:", cos_sim)

cos_sim with best submission: 0.9999823181849743
